In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import models
from keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Data understanding

### Load data and visualize it

In [ ]:
df = pd.read_csv('generated_data.csv')
df

In [ ]:
df.plot(title='Plot of the data set', ylabel='Value ($^\circ{}$C/psi/%)', xlabel='Data point index')

In [ ]:
df['temperature'].plot(title='Plot of temperature', xlabel='Data point index', ylabel='Temperature ($^\circ{}$C)', legend=True)

In [ ]:
plt.scatter(df[df['temperature_status']=='normal'].index, df[df['temperature_status']=='normal']['temperature'], label='Normal')
plt.scatter(df[df['temperature_status']!='normal'].index, df[df['temperature_status']!='normal']['temperature'], label='Anomalous')
plt.title('Normal and anomalous temperature data points')
plt.xlabel('Data point index')
plt.ylabel('Temperature ($^\circ{}$C)')
plt.legend(loc='upper right')

# Data preprocessing

### Split data into training and testing sets

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

### Normalize data using Min-Max normalization

In [ ]:
temp_scaler = MinMaxScaler()
pressure_scaler = MinMaxScaler()
humidity_scaler = MinMaxScaler()
df_train['temperature_norm'] = temp_scaler.fit_transform(df_train[['temperature']])
df_train['pressure_norm'] = pressure_scaler.fit_transform(df_train[['pressure']])
df_train['humidity_norm'] = humidity_scaler.fit_transform(df_train[['humidity']])
df_test['temperature_norm'] = temp_scaler.transform(df_test[['temperature']])
df_test['pressure_norm'] = pressure_scaler.transform(df_test[['pressure']])
df_test['humidity_norm'] = humidity_scaler.transform(df_test[['humidity']])

### Split data into normal and anomalous data sets

In [ ]:
df_train_temp_normal = df_train[df_train['temperature_status'] == 'normal'][['timestamp', 'temperature', 'temperature_norm']]
df_test_temp_normal = df_test[df_test['temperature_status'] == 'normal'][['timestamp', 'temperature', 'temperature_norm']]
df_temp_anomal_low = df_test[(df_test['temperature_status'] != 'normal') & (df_test['temperature'] < 26)][['timestamp', 'temperature', 'temperature_norm']]
df_temp_anomal_high = df_test[(df_test['temperature_status'] != 'normal') & (df_test['temperature'] > 26)][['timestamp', 'temperature', 'temperature_norm']]

df_train_pressure_normal = df_train[df_train['pressure_status'] == 'normal'][['timestamp', 'pressure', 'pressure_norm']]
df_test_pressure_normal = df_test[df_test['pressure_status'] == 'normal'][['timestamp', 'pressure', 'pressure_norm']]
df_pressure_anomal_low = df_test[(df_test['pressure_status'] != 'normal') & (df_test['pressure'] < 100)][['timestamp', 'pressure', 'pressure_norm']]
df_pressure_anomal_high = df_test[(df_test['pressure_status'] != 'normal') & (df_test['pressure'] > 100)][['timestamp', 'pressure', 'pressure_norm']]

df_train_humidity_normal = df_train[df_train['humidity_status'] == 'normal'][['timestamp', 'humidity', 'humidity_norm']]
df_test_humidity_normal = df_test[df_test['humidity_status'] == 'normal'][['timestamp', 'humidity', 'humidity_norm']]
df_humidity_anomal_low = df_test[(df_test['humidity_status'] != 'normal') & (df_test['humidity'] < 66)][['timestamp', 'humidity', 'humidity_norm']]
df_humidity_anomal_high = df_test[(df_test['humidity_status'] != 'normal') & (df_test['humidity'] > 66)][['timestamp', 'humidity', 'humidity_norm']]

In [ ]:
df_temp_anomal_high.plot()

# Modelling

### Divide datasets into signals with length defined by lookback

In [ ]:
lookback = 5

In [ ]:
def df_to_input(df):
    t = np.array(df)
    X = []
    for i in range(len(t) - lookback):
        tX = []
        for j in range(i, i+lookback):
            tX.append([t[j]])
        X.append(tX)
    return np.array(X)

In [ ]:
X = df_to_input(df_train_temp_normal['temperature_norm'])
X

### Define the model

In [ ]:
def get_model():
    inp = Input(shape=(lookback,1))
    # Encoder
    x = LSTM(8, activation='relu', input_shape=(lookback, 1), return_sequences=True)(inp)
    x = LSTM(4, activation='relu', return_sequences=False)(x)
    # Decoder
    x = RepeatVector(lookback)(x)
    x = LSTM(4, activation='relu', return_sequences=True)(x)
    x = LSTM(8, activation='relu', return_sequences=True)(x)
    x = TimeDistributed(Dense(1))(x)
    model = models.Model(
        inputs = inp,
        outputs=x
    )
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
model = get_model()
model.summary()

### Train the model

In [ ]:
validation = df_to_input(df_test_temp_normal['temperature_norm'])
history = model.fit(x=X, y=X, epochs=100, validation_data=(validation, validation))

### Load the pretrained model with good performance

In [ ]:
model = tf.keras.models.load_model('model/8_4_96.6')

### Plot the loss and metrics for the trained model

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.title('Training and validation loss for LSTM Autoencoder')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.legend()
plt.title('Training and validation Mean Absolute Error for LSTM Autoencoder')
plt.ylabel('MAE')
plt.xlabel('Epoch')


# Evaluation

### Define the threshold

In [ ]:
predictions = model.predict(X)
losses = tf.keras.losses.mae(predictions, X)
mean, std = np.mean(losses), np.std(losses)
threshold = mean + std*1
print(f'{mean=}, {std=}, {threshold=}')

### Evaluate the model on the test data set

In [ ]:
def evaluate(X, threshold=threshold):
    X = df_to_input(X)
    predictions = model.predict(X, verbose=0)
    test_losses = tf.keras.losses.mae(predictions, X)
    test_loss = np.mean(test_losses, axis=1)
    normal, anomaly = len(test_loss[test_loss <= threshold]), len(test_loss[test_loss > threshold])
    return {'normal': normal, 'anomaly': anomaly, 'mean': np.mean(test_loss), 'std': np.std(test_loss)}

#### Temperature

In [ ]:
norm = evaluate(df_test_temp_normal['temperature_norm'], threshold)
high = evaluate(df_temp_anomal_high['temperature_norm'], threshold)
low = evaluate(df_temp_anomal_low['temperature_norm'], threshold)
TN = norm['normal']
TP =  high['anomaly'] + low['anomaly']
FN =  + high['normal'] + low['normal']
FP = norm['anomaly']
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2*precision * recall / (precision + recall)
print(f'{TP=}, {TN=}, {FP=}, {FN=}, {accuracy=:.4f}, {precision=:.4f}, {recall=:.3f}, {F1=:.4f}')

#### Pressure

In [ ]:
norm = evaluate(df_test_pressure_normal['pressure_norm'], threshold)
high = evaluate(df_pressure_anomal_high['pressure_norm'], threshold)
low = evaluate(df_pressure_anomal_low['pressure_norm'], threshold)
TN = norm['normal']
TP =  high['anomaly'] + low['anomaly']
FN =  + high['normal'] + low['normal']
FP = norm['anomaly']
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2*precision * recall / (precision + recall)
print(f'{TP=}, {TN=}, {FP=}, {FN=}, {accuracy=:.4f}, {precision=:.4f}, {recall=:.3f}, {F1=:.4f}')

#### Humidity

In [ ]:
norm = evaluate(df_test_humidity_normal['humidity_norm'], threshold)
high = evaluate(df_humidity_anomal_high['humidity_norm'], threshold)
low = evaluate(df_humidity_anomal_low['humidity_norm'], threshold)
TN = norm['normal']
TP =  high['anomaly'] + low['anomaly']
FN =  + high['normal'] + low['normal']
FP = norm['anomaly']
accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2*precision * recall / (precision + recall)
print(f'{TP=}, {TN=}, {FP=}, {FN=}, {accuracy=:.4f}, {precision=:.4f}, {recall=:.3f}, {F1=:.4f}')